In [1]:
!pip install pygame
!pip install neat-python

In [2]:
import os
import random
import math
import pygame
from os import listdir
from os.path import isfile, join
import neat
pygame.init()

pygame.display.set_caption("Platformer")

WIDTH, HEIGHT = 1000, 800
FPS = 60
PLAYER_VEL = 5

window = pygame.display.set_mode((WIDTH, HEIGHT))


pygame 2.5.2 (SDL 2.28.3, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def flip(sprites):
    return [pygame.transform.flip(sprite, True, False) for sprite in sprites]


def load_sprite_sheets(dir1, dir2, width, height, direction=False):
    path = join("assets", dir1, dir2)
    images = [f for f in listdir(path) if isfile(join(path, f))]

    all_sprites = {}

    for image in images:
        sprite_sheet = pygame.image.load(join(path, image)).convert_alpha()

        sprites = []
        for i in range(sprite_sheet.get_width() // width):
            surface = pygame.Surface((width, height), pygame.SRCALPHA, 32)
            rect = pygame.Rect(i * width, 0, width, height)
            surface.blit(sprite_sheet, (0, 0), rect)
            sprites.append(pygame.transform.scale2x(surface))

        if direction:
            all_sprites[image.replace(".png", "") + "_right"] = sprites
            all_sprites[image.replace(".png", "") + "_left"] = flip(sprites)
        else:
            all_sprites[image.replace(".png", "")] = sprites

    return all_sprites


In [4]:
class Player(pygame.sprite.Sprite):
    COLOR = (255, 0, 0)
    GRAVITY = 1
    SPRITES = load_sprite_sheets("MainCharacters", "MaskDude", 32, 32, True)
    ANIMATION_DELAY = 3

    def __init__(self, x, y, width, height):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.x_vel = 0
        self.y_vel = 0
        self.mask = None
        self.direction = "left"
        self.animation_count = 0
        self.fall_count = 0
        self.jump_count = 0
        self.hit = False
        self.hit_count = 0

    def jump(self):
        self.y_vel = -self.GRAVITY * 8
        self.animation_count = 0
        self.jump_count += 1
        if self.jump_count == 1:
            self.fall_count = 0

    def move(self, dx, dy):
        self.rect.x += dx
        self.rect.y += dy

    def make_hit(self):
        self.hit = True

    def move_left(self, vel):
        self.x_vel = -vel
        if self.direction != "left":
            self.direction = "left"
            self.animation_count = 0

    def move_right(self, vel):
        self.x_vel = vel
        if self.direction != "right":
            self.direction = "right"
            self.animation_count = 0

    def loop(self, fps):
        self.y_vel += min(1, (self.fall_count / fps) * self.GRAVITY)
        self.move(self.x_vel, self.y_vel)

        if self.hit:
            self.hit_count += 1
        if self.hit_count > fps * 2:
            self.hit = False
            self.hit_count = 0

        self.fall_count += 1
        self.update_sprite()

    def landed(self):
        self.fall_count = 0
        self.y_vel = 0
        self.jump_count = 0

    def hit_head(self):
        self.count = 0
        self.y_vel *= -1

    def update_sprite(self):
        sprite_sheet = "idle"
        if self.hit:
            sprite_sheet = "hit"
        elif self.y_vel < 0:
            if self.jump_count == 1:
                sprite_sheet = "jump"
            elif self.jump_count == 2:
                sprite_sheet = "double_jump"
        elif self.y_vel > self.GRAVITY * 2:
            sprite_sheet = "fall"
        elif self.x_vel != 0:
            sprite_sheet = "run"

        sprite_sheet_name = sprite_sheet + "_" + self.direction
        sprites = self.SPRITES[sprite_sheet_name]
        sprite_index = (self.animation_count //
                        self.ANIMATION_DELAY) % len(sprites)
        self.sprite = sprites[sprite_index]
        self.animation_count += 1
        self.update()

    def update(self):
        self.rect = self.sprite.get_rect(topleft=(self.rect.x, self.rect.y))
        self.mask = pygame.mask.from_surface(self.sprite)

    def draw(self, win, offset_x):
        win.blit(self.sprite, (self.rect.x - offset_x, self.rect.y))

In [5]:
def get_block(size):
    path = join("assets", "Terrain", "Terrain.png")
    image = pygame.image.load(path).convert_alpha()
    surface = pygame.Surface((size, size), pygame.SRCALPHA, 32)
    rect = pygame.Rect(96, 0, size, size)
    surface.blit(image, (0, 0), rect)
    return pygame.transform.scale2x(surface)




class Object(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height, name=None):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.image = pygame.Surface((width, height), pygame.SRCALPHA)
        self.width = width
        self.height = height
        self.name = name

    def draw(self, win, offset_x):
        win.blit(self.image, (self.rect.x - offset_x, self.rect.y))


class Block(Object):
    def __init__(self, x, y, size):
        super().__init__(x, y, size, size)
        block = get_block(size)
        self.image.blit(block, (0, 0))
        self.mask = pygame.mask.from_surface(self.image)

In [6]:
class Fire(Object):
    ANIMATION_DELAY = 3

    def __init__(self, x, y, width, height):
        super().__init__(x, y, width, height, "fire")
        self.fire = load_sprite_sheets("Traps", "Fire", width, height)
        self.image = self.fire["off"][0]
        self.mask = pygame.mask.from_surface(self.image)
        self.animation_count = 0
        self.animation_name = "off"

    def on(self):
        self.animation_name = "on"

    def off(self):
        self.animation_name = "off"

    def loop(self):
        sprites = self.fire[self.animation_name]
        sprite_index = (self.animation_count //
                        self.ANIMATION_DELAY) % len(sprites)
        self.image = sprites[sprite_index]
        self.animation_count += 1

        self.rect = self.image.get_rect(topleft=(self.rect.x, self.rect.y))
        self.mask = pygame.mask.from_surface(self.image)

        if self.animation_count // self.ANIMATION_DELAY > len(sprites):
            self.animation_count = 0

In [7]:



def get_background(name):
    image = pygame.image.load(join("assets", "Background", name))
    _, _, width, height = image.get_rect()
    tiles = []

    for i in range(WIDTH // width + 1):
        for j in range(HEIGHT // height + 1):
            pos = (i * width, j * height)
            tiles.append(pos)

    return tiles, image


def draw(window, background, bg_image, players, objects, offset_x, playerRemains, generation, curr_score, high_score, time_limit):
    for tile in background:
        window.blit(bg_image, tile)

    text = font.render("players remaining: " + str(playerRemains), True, (0, 0, 0))
    text_rect = text.get_rect()
    text_rect.center = (WIDTH/2, 130)
    window.blit(text, text_rect)

    Gtext = font.render("Generation: " + str(generation), True, (0, 0, 0))
    Gtext_rect = Gtext.get_rect()
    Gtext_rect.center = (WIDTH/2, 100)
    window.blit(Gtext, Gtext_rect)

    curr_score = font.render("Generation High Score: " + str(round(curr_score, 2)), True, (0, 0, 0))
    curr_rect = curr_score.get_rect()
    curr_rect.center = (WIDTH/2, 160)
    window.blit(curr_score, curr_rect)

    highText = font.render("Highest Score Overall: " + str(round(high_score, 2)), True, (0, 0, 0))
    high_rect = highText.get_rect()
    high_rect.center = (WIDTH/2, 190)
    window.blit(highText, high_rect)

    timeText = font.render("Time Limit: " + str(time_limit), True, (0, 0, 0))
    time_rect = timeText.get_rect()
    time_rect.center = (WIDTH/2, 210)
    window.blit(timeText, time_rect)

    for obj in objects:
        obj.draw(window, offset_x)
    
    for player in players:
        player.draw(window, offset_x)

    pygame.display.update()


def handle_vertical_collision(player, objects, dy):
    collided_objects = []
    for obj in objects:
        if pygame.sprite.collide_mask(player, obj):
            if dy > 0:
                player.rect.bottom = obj.rect.top
                player.landed()
            elif dy < 0:
                player.rect.top = obj.rect.bottom
                player.hit_head()

            collided_objects.append(obj)

    return collided_objects


def collide(player, objects, dx):
    player.move(dx, 0)
    player.update()
    collided_object = None
    for obj in objects:
        if pygame.sprite.collide_mask(player, obj):
            collided_object = obj
            break

    player.move(-dx, 0)
    player.update()
    return collided_object


def handle_move(player, objects, action):
    player.x_vel = 0
    collide_left = collide(player, objects, -PLAYER_VEL * 2)
    collide_right = collide(player, objects, PLAYER_VEL * 2)

    if action == "left" and not collide_left:
        player.move_left(PLAYER_VEL)
    if action == "right" and not collide_right:
        player.move_right(PLAYER_VEL)

    vertical_collide = handle_vertical_collision(player, objects, player.y_vel)
    to_check = [collide_left, collide_right, *vertical_collide]

    for obj in to_check:
        if obj and obj.name == "fire":
            player.make_hit()

In [8]:
clock = pygame.time.Clock()
background, bg_image = get_background("Blue.png")
font = pygame.font.SysFont("Arial", 30)

block_size = 96


fire = Fire(170, 640, 16, 32)
fire2 = Fire(550, 640 - block_size, 16, 32)
fire3 = Fire(750, 640, 16, 32)
fire4 = Fire(1000, 640, 16, 32)
fire5 = Fire(1500, 640, 16, 32)
fire.on()
fire2.on()
fire3.on()
fire4.on()
fire5.on()

floor = [Block(i * block_size, HEIGHT - block_size, block_size)
             for i in range(-WIDTH // block_size, (WIDTH * 2) // block_size)]
#floor = Block(100, HEIGHT - block_size, block_size)

'''win_platform = [Block(i * block_size, HEIGHT - block_size * 6, block_size)
                    for i in range( 10, (WIDTH * 2) // block_size)]'''
    
objects = [*floor, Block(500, HEIGHT - block_size * 2, block_size), fire, fire2, fire3, fire4, fire5]
#Block(block_size * 3 + 500, HEIGHT - block_size * 4, block_size)
generation = 0
high_score = -1000
time_limit = 100




In [9]:
def eval_genomes(genomes, config):

    offset_x = 0
    scroll_area_width = 200
    #NEAT
    nets = []
    ge = []
    dis = []
        

    #player = Player(100, 100, 50, 50)
    players = []

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        players.append(Player(10, 640, 50, 50))
        g.fitness = 0
        ge.append(g)
        dis.append(10)

    run = True
    time = 0
    global generation, high_score, time_limit
    generation += 1
    if generation % 5 == 0:
        time_limit += 10
    while run:
        curr_high_score = -1000
        clock.tick(FPS)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
                break

        for x, player in enumerate(players):
            ge[x].fitness -= 0.08

            #output = nets[x].activate((player.rect.x, player.rect.y, (1 if player.jump_count < 2 else 0),  abs(player.rect.x - 100), abs(player.rect.y - (HEIGHT - block_size)), abs(player.rect.x - 500), abs(player.rect.y - (HEIGHT - block_size * 2)), abs(player.rect.x - (block_size * 3 + 500)), abs(player.rect.y - (HEIGHT - block_size * 4)), abs(player.rect.x - 1725), abs(player.rect.y - 640)))
            output = nets[x].activate((player.rect.x, player.rect.y, (1 if player.jump_count < 2 else 0),  abs(player.rect.x - 170), abs(player.rect.y - 640), abs(player.rect.x - 550), abs(player.rect.y - 554), abs(player.rect.x - 750), abs(player.rect.x - 640), abs(player.rect.x - 1000),  abs(player.rect.y - 640), abs(player.rect.x - 1500), abs(player.rect.y - 640), abs(player.rect.x - 1815)))
            i =  output.index(max(output))
            if i == 0:
                #player.move_left(PLAYER_VEL)
                action = "left"
            elif i == 1:
                #player.move_right(PLAYER_VEL)
                action = "right"
            else:
                if player.jump_count < 2:
                    player.jump()
                action = ""

            '''if ((player.rect.x >= 1725) and (player.rect.y == 160)):
                ge[x].fitness += 5'''

            if (player.rect.x >= 1800):
                ge[x].fitness += 50
            elif dis[x] >= player.rect.x:
                ge[x].fitness -= 0.5
            elif dis[x] < player.rect.x:
                ge[x].fitness += 0.075

            if player.rect.x > 1500:
                ge[x].fitness += 0.05
            elif player.rect.x > 1000:
                ge[x].fitness += 0.04
            elif player.rect.x > 750:
                ge[x].fitness += 0.03
            elif player.rect.x > 550:
                ge[x].fitness += 0.02
            elif player.rect.x > 170:
                ge[x].fitness += 0.01


            '''
            elif player.rect.x >= 1300:
                ge[x].fitness += 0.5
            elif player.rect.x >= 700:
                ge[x].fitness += 0.2'''
            
    
            if ((player.rect.right - offset_x >= WIDTH - scroll_area_width) and player.x_vel > 0) or (
                (player.rect.left - offset_x <= scroll_area_width) and player.x_vel < 0):
                offset_x += player.x_vel
    
            player.loop(FPS)
            handle_move(player, objects, action)

            if curr_high_score < ge[x].fitness:
                curr_high_score = ge[x].fitness

            if player.hit or player.rect.y >= 1200 or player.rect.x < 0 or time > time_limit:
                ge[x].fitness += -500
                players.pop(x)
                nets.pop(x)
                ge.pop(x)
                dis.pop(x)

            if player.rect.x >= 1815:
                players.pop(x)
                nets.pop(x)
                ge.pop(x)
                dis.pop(x)

            
        fire.loop()
        fire2.loop()
        fire3.loop()
        fire4.loop()
        fire5.loop()

        if curr_high_score > high_score:
            high_score = curr_high_score
        draw(window, background, bg_image, players, objects, offset_x, len(players), generation, curr_high_score, high_score, time_limit)
        time += 1
        if len(players) == 0:
            run = False
            break


    #pygame.quit()
    #quit()

In [10]:
def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)
    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #p.add_reporter(neat.Checkpointer(5))

    # Run for up to 50 generations.
    winner = p.run(eval_genomes, 500)

    # show final stats
    print('\nBest genome:\n{!s}'.format(winner))

In [ ]:
config_path = os.path.join("config.txt")
run(config_path)
pygame.quit()
quit()


 ****** Running generation 0 ****** 

Population's average fitness: -508.80410 stdev: 18.45861
Best fitness: -501.25500 - size: (3, 42) - species 1 - id 12
Average adjusted fitness: 0.870
Mean genetic distance 1.209, standard deviation 0.272
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50   -501.3    0.870     0
Total extinctions: 0
Generation time: 1.809 sec

 ****** Running generation 1 ****** 

Population's average fitness: -507.27650 stdev: 17.03217
Best fitness: -501.25500 - size: (3, 42) - species 1 - id 12
Average adjusted fitness: 0.896
Mean genetic distance 1.267, standard deviation 0.315
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    50   -501.3    0.896     1
Total extinctions: 0
Generation time: 1.845 sec (1.827 average)

 ****** Running generation 2 ****** 

Population's average fitness: -50

In [ ]:
pygame.quit()
quit()

In [ ]:
#!git clone https://github.com/techwithtim/Python-Platformer.git